In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
%load_ext autoreload

%autoreload 2

In [7]:
import math
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread, imsave, figure
import pandas as pd
import numpy as np
import os
import sys
from skimage.color import rgb2gray
from skimage.transform import rescale, resize


#Keras:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

#Custom Functions:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

from imports.utils.log_progress import log_progress
from imports.utils.visualization import Visualize, Evaluate
from imports.utils.enums import DATA_BASE_PATH, SHAPE
from imports.models.u_net import get_unet
from imports.utils.utils import rle_encode, rle_decode

DATA_IMAGE_PATH = DATA_BASE_PATH + '/Images'
DATA_MASK_PATH = DATA_BASE_PATH + '/Masks'

In [4]:
train_df = pd.read_pickle(DATA_BASE_PATH+'/train_df')
val_df = pd.read_pickle(DATA_BASE_PATH+'/val_df')
test_df = pd.read_pickle(DATA_BASE_PATH+'/test_df')

print("Number of training Samples:", len(train_df))
print("Number of validation Samples:", len(val_df))
print("Number of test Samples:", len(test_df))

Number of training Samples: 354
Number of validation Samples: 102
Number of test Samples: 51


#### Models:

| Model | Description | Filename |
|:------|------|--|
|   UNet_Mask  | Unet- Model trained on Handlabeled Mask-Data 'train_df'| unet1024_60epochs.h5 |
|   UNet_Circle  | Unet- Model trained on Handlabeled Mask-Data 'train_df'| - |

In [5]:
from imports.models.losses import bce_dice_loss, dice_coeff
from keras.models import load_model, model_from_json

import keras.losses
keras.losses.custom_loss = bce_dice_loss

unet_mask = load_model('../saved_models/unet/unet1024_60epochs.h5', custom_objects={'bce_dice_loss': bce_dice_loss,'dice_coeff':dice_coeff})

In [20]:
ev_params = {'df':test_df,
             'data_set':'test',
              'model':unet_mask}

ev = Evaluate(**ev_params)

TypeError: __init__() got an unexpected keyword argument 'df'

In [19]:
ev.get_dice_coeff()

AttributeError: type object 'Visualize' has no attribute '_Evaluate__load_data'